In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, explained_variance_score
import sklearn.metrics

import zipfile
import requests
import os
import shutil

%matplotlib inline

In [127]:
#Get data
filestring = './data/anc_out.csv'
df = pd.read_csv(filestring)

df1 = df #copy it

df

,index_left,Util_Indx_BBL,countBBL,countIssued,month,pct_metro_coverage,NAME_ANC 1A,NAME_ANC 1B,NAME_ANC 1C,NAME_ANC 1D,...,NAME_ANC 7B,NAME_ANC 7C,NAME_ANC 7D,NAME_ANC 7E,NAME_ANC 7F,NAME_ANC 8A,NAME_ANC 8B,NAME_ANC 8C,NAME_ANC 8D,NAME_ANC 8E
0,0,0.054171,200.0,1.0,2012.01,0.001354,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.030303,66.0,0.0,2012.01,0.050505,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,0.068413,235.0,3.0,2012.01,0.727116,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0.080778,748.0,7.0,2012.01,0.785084,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.107343,421.0,1.0,2012.01,0.989271,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0.025030,21.0,0.0,2012.01,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0.028558,178.0,8.0,2012.01,0.197016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0.047672,301.0,3.0,2012.01,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0.026827,116.0,4.0,2012.01,0.387604,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0.026550,203.0,1.0,2012.01,0.074549,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
df = df1
df['y']= df['countIssued'].shift(-80)
df = df.dropna()
df = df.drop('index_left', axis= 1)
df = df.fillna(0)
np.any(np.isnan(df))

False

In [129]:
np.all(np.isfinite(df))

True

In [130]:
df.columns

Index(['Util_Indx_BBL', 'countBBL', 'countIssued', 'month',
       'pct_metro_coverage', 'NAME_ANC 1A', 'NAME_ANC 1B', 'NAME_ANC 1C',
       'NAME_ANC 1D', 'NAME_ANC 2A', 'NAME_ANC 2B', 'NAME_ANC 2C',
       'NAME_ANC 2D', 'NAME_ANC 2E', 'NAME_ANC 2F', 'NAME_ANC 3B',
       'NAME_ANC 3C', 'NAME_ANC 3D', 'NAME_ANC 3E', 'NAME_ANC 3F',
       'NAME_ANC 3G', 'NAME_ANC 4A', 'NAME_ANC 4B', 'NAME_ANC 4C',
       'NAME_ANC 4D', 'NAME_ANC 5A', 'NAME_ANC 5B', 'NAME_ANC 5C',
       'NAME_ANC 5D', 'NAME_ANC 5E', 'NAME_ANC 6A', 'NAME_ANC 6B',
       'NAME_ANC 6C', 'NAME_ANC 6D', 'NAME_ANC 6E', 'NAME_ANC 7B',
       'NAME_ANC 7C', 'NAME_ANC 7D', 'NAME_ANC 7E', 'NAME_ANC 7F',
       'NAME_ANC 8A', 'NAME_ANC 8B', 'NAME_ANC 8C', 'NAME_ANC 8D',
       'NAME_ANC 8E', 'y'],
      dtype='object')

In [131]:
df[['y', 'countIssued','NAME_ANC 1D', 'month', 
    'countBBL', 'pct_metro_coverage']][df['NAME_ANC 1D'] == 1]

,y,countIssued,NAME_ANC 1D,month,countBBL,pct_metro_coverage
1,5.0,0.0,1,2012.01,66.0,0.050505
41,2.0,0.0,1,2012.02,63.0,0.050505
81,7.0,5.0,1,2012.03,66.0,0.050505
121,4.0,2.0,1,2012.04,66.0,0.050505
161,1.0,7.0,1,2012.05,72.0,0.050505
201,1.0,4.0,1,2012.06,72.0,0.050505
241,1.0,1.0,1,2012.07,75.0,0.050505
281,10.0,1.0,1,2012.08,78.0,0.050505
321,3.0,1.0,1,2012.09,80.0,0.050505
361,6.0,10.0,1,2012.10,87.0,0.050505


In [141]:
data = df
data = data.reset_index()
cv = KFold(n_splits=10, shuffle=False, random_state=0)
for train_index, test_index in cv.split(data):
    ## Define training and test sets
    X_train = data.drop(['Util_Indx_BBL','index','y'], axis=1)
    
    X_train = X_train.loc[train_index]
    y_train = data['y']
    y_train = y_train.loc[train_index]
    X_test  = data.loc[test_index]
    X_test  = X_test.drop(['Util_Indx_BBL','index','y'], axis=1)
    y_test  = data['y']
    y_test  = y_test.loc[test_index]
    
       
    ## Fit model
    clf = RandomForestRegressor(n_estimators=X_train.shape[0], max_depth=10)
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    
    ## Compare to actual outcomes and return precision
    print('R2_Score: '+str(sklearn.metrics.regression.r2_score(y_test, predicted)))
    
    print('stderr: ' + str(sklearn.metrics.regression.mean_squared_error(y_test, predicted)))

R2_Score: 0.260584236397
stderr: 91.3580157202
R2_Score: 0.190677644435
stderr: 124.895232181
R2_Score: 0.351421642294
stderr: 30.7434624742
R2_Score: 0.338856130575
stderr: 98.3036325501
R2_Score: 0.225585386983
stderr: 245.892854802
R2_Score: -0.0965066211235
stderr: 108.366732392
R2_Score: 0.568428832943
stderr: 70.41666032
R2_Score: -0.06338105911
stderr: 404.442634614
R2_Score: 0.543033417396
stderr: 1062.99262673
R2_Score: 0.646271855092
stderr: 536.172524638


In [139]:
print([[]])

24       17.0
25       10.0
67        0.0
84       34.0
86       12.0
91        7.0
94       10.0
99       10.0
130      16.0
136       4.0
146       4.0
151       9.0
166       4.0
168       3.0
176       4.0
180      11.0
197       5.0
199      10.0
201       1.0
207       3.0
209       1.0
216       3.0
221       2.0
256       1.0
257       0.0
273       0.0
274       1.0
275       0.0
277       0.0
291      10.0
        ...  
2080     87.0
2081     28.0
2084     64.0
2089     62.0
2091     45.0
2105     37.0
2120     96.0
2121     34.0
2135     23.0
2146     64.0
2148     25.0
2159     85.0
2163    241.0
2169     60.0
2171     39.0
2176     20.0
2187     21.0
2209     53.0
2215     15.0
2217     66.0
2219     61.0
2221     61.0
2222     67.0
2239     61.0
2245      5.0
2251     31.0
2286     54.0
2293     34.0
2294     23.0
2302     88.0
Name: y, Length: 232, dtype: float64 /n [  11.14846743    7.65038314    1.2940613    43.56561303   14.1532567
    4.43534483    3.31082375   25.76